In [1]:
import gmsh

In [19]:
gmsh.initialize()
gmsh.model.add("mesh_with_occt")

In [20]:
inner_radius1 = 1/3
inner_radius2 = 2/3
outer_radius = 1.

outer_sphere = gmsh.model.occ.addSphere(0, 0, 0, outer_radius)
inner_sphere2 = gmsh.model.occ.addSphere(0, 0, 0, inner_radius2)
inner_sphere1 = gmsh.model.occ.addSphere(0, 0, 0, inner_radius1)

In [21]:
cut_result1 = gmsh.model.occ.cut([(3,outer_sphere)],[(3,inner_sphere2)], removeTool=False)
print("Cut Result 1", cut_result1)

Cut Result 1 ([(3, 1)], [[(3, 1)], []])                                                                                   


In [22]:
gmsh.model.occ.synchronize()

In [23]:
volEntities = gmsh.model.getEntities(3)
print("Volumes:", volEntities)
shell_volume1 = None
for (dim, tag) in volEntities:
    if tag != inner_sphere1 and tag != inner_sphere2:
        shell_volume1 = tag
        break
if shell_volume1 is None:
    raise RuntimeError("Could not identify the shell volume.")

Volumes: [(3, 1), (3, 2), (3, 3)]


In [24]:
cut_result2 = gmsh.model.occ.cut([(3,inner_sphere2)],[(3,inner_sphere1)], removeTool=False)
print("Cut Result 2", cut_result2)

Cut Result 2 ([(3, 2)], [[(3, 2)], []])les                                                                                
                                                                                

In [25]:
gmsh.model.occ.synchronize()

In [26]:
volEntities = gmsh.model.getEntities(3)
print("Volumes:", volEntities)
shell_volume2 = None
for (dim, tag) in volEntities:
    if tag != inner_sphere1 and tag != shell_volume1:
        shell_volume2 = tag
        break
if shell_volume2 is None:
    raise RuntimeError("Could not identify the shell volume.")

Volumes: [(3, 1), (3, 2), (3, 3)]


In [27]:
print(inner_sphere1)
print(shell_volume1)
print(shell_volume2)

3
1
2


In [28]:
# Define physical groups for the inner sphere and shell
gmsh.model.addPhysicalGroup(3, [inner_sphere1], tag=1)
gmsh.model.setPhysicalName(3, 1, "Inner")
gmsh.model.addPhysicalGroup(3, [shell_volume1], tag=3)
gmsh.model.setPhysicalName(3, 2, "Shell 1")
gmsh.model.addPhysicalGroup(3, [shell_volume2], tag=2)
gmsh.model.setPhysicalName(3, 2, "Shell2")
# Create a MathEval field to control the mesh size based on z.
fieldTag = gmsh.model.mesh.field.add("MathEval")

In [29]:
def Mesh(h=0.1, alg=6):
    gmsh.option.setNumber("Mesh.MeshSizeExtendFromBoundary", 0)
    gmsh.option.setNumber("Mesh.MeshSizeFromPoints", 0)
    gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 0)
    gmsh.option.setNumber("Mesh.MeshSizeMax", h)

    # Export Mesh to specified version for OpenSn
    gmsh.option.setNumber("Mesh.MshFileVersion",2.2)
    gmsh.option.setNumber("Mesh.Algorithm", alg)

    gmsh.model.mesh.generate(3)
    gmsh.write("problem_one.msh")

In [30]:
gmsh.model.mesh.field.setString(fieldTag, "F", "0.99")
# Use this field as the background mesh field.
gmsh.model.mesh.field.setAsBackgroundMesh(fieldTag)
# Generate the 3D mesh
Mesh()

Info    : Meshing 1D...
Info    : [ 20%] Meshing curve 5 (Circle)
Info    : [ 50%] Meshing curve 8 (Circle)
Info    : [ 80%] Meshing curve 11 (Circle)
Info    : Done meshing 1D (Wall 0.000820797s, CPU 0.001205s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 2 (Sphere, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 3 (Sphere, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 4 (Sphere, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.088087s, CPU 0.080112s)
Info    : Meshing 3D...
Info    : 3D Meshing 3 volumes with 1 connected component
Info    : Tetrahedrizing 2399 nodes...
Info    : Done tetrahedrizing 2407 nodes (Wall 0.0180729s, CPU 0.01672s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.0390895s, CPU 0.027407s)
Info    : Found volume 3
Info    : Found volume 2
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst 

In [31]:
# Save mesh in VTK format
gmsh.write("vtk_meshes/three_spheres.vtk")

Info    : Writing 'vtk_meshes/three_spheres.vtk'...
Info    : Done writing 'vtk_meshes/three_spheres.vtk'


In [18]:
gmsh.finalize()